In [1]:
#
# Working through
# https://github.com/salvadorgarciamunoz/kipet/blob/master/kipet/examples/Ex_7_conc_input_conf_k_aug.py
#

In [2]:
from kipet import KipetModel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

    Set objects from pyomo.core.base.set or pyomo.core.  (deprecated in 5.7)
    (called from <frozen importlib._bootstrap>:219)
_SetProduct not found


In [3]:
filename = '/home/paperspace/learn_kipet/my_data_sets/ABC_data.csv'
C_data = pd.read_csv(filename, index_col = 0)
C_data.columns = ['A', 'B', 'C']
C_data

,A,B,C
t,,,
0.000,0.957,-0.031,-0.015
0.263,0.557,0.330,0.044
0.526,0.342,0.512,0.156
0.789,0.224,0.499,0.310
1.053,0.123,0.428,0.454
1.316,0.079,0.396,0.556
1.579,0.035,0.303,0.651
1.842,0.029,0.287,0.658
2.105,0.025,0.221,0.750


In [4]:
mdl = KipetModel()

r1 = mdl.new_reaction('rxn1')

r1.add_component('A', state = 'concentration', init = 1.0, variance = 1)
r1.add_component('B', state = 'concentration', init = 0.0, variance = 1)
r1.add_component('C', state = 'concentration', init = 0.0, variance = 1)

r1.add_parameter('k1', init = 5.0, bounds = (0.0, 10.0))
r1.add_parameter('k2', init = 5.0, bounds = (0.0, 10.0))

In [5]:
# define explicit system of ODEs
def rule_odes(m,t):
    exprs = dict()
    exprs['A'] = -m.P['k1']*m.Z[t,'A']
    exprs['B'] = m.P['k1']*m.Z[t,'A']-m.P['k2']*m.Z[t,'B']
    exprs['C'] = m.P['k2']*m.Z[t,'B']
    return exprs

r1.add_equations(rule_odes)
r1.set_times(0.0, 5.0)

In [6]:
r1.add_dataset('C_data', category = 'concentration', data = C_data)

In [7]:
r1.settings.collocation.nfe = 50
r1.settings.collocation.ncp = 3
r1.settings.parameter_estimator['solver'] = 'k_aug'
r1.settings.solver.linear_solver = 'ma27'

In [8]:
r1.create_pyomo_model()

Warning! Since Cm-matrix contains negative values Kipet is assuming a derivative of Cm has been inputted
times are: <class 'tuple'>
0.0 5.0


In [9]:
r1.model.pprint()

14 Set Declarations
    Cm_indx : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   60 : {(0.0, 'A'), (0.0, 'B'), (0.0, 'C'), (0.263, 'A'), (0.263, 'B'), (0.263, 'C'), (0.526, 'A'), (0.526, 'B'), (0.526, 'C'), (0.789, 'A'), (0.789, 'B'), (0.789, 'C'), (1.053, 'A'), (1.053, 'B'), (1.053, 'C'), (1.316, 'A'), (1.316, 'B'), (1.316, 'C'), (1.579, 'A'), (1.579, 'B'), (1.579, 'C'), (1.842, 'A'), (1.842, 'B'), (1.842, 'C'), (2.105, 'A'), (2.105, 'B'), (2.105, 'C'), (2.368, 'A'), (2.368, 'B'), (2.368, 'C'), (2.632, 'A'), (2.632, 'B'), (2.632, 'C'), (2.895, 'A'), (2.895, 'B'), (2.895, 'C'), (3.158, 'A'), (3.158, 'B'), (3.158, 'C'), (3.421, 'A'), (3.421, 'B'), (3.421, 'C'), (3.684, 'A'), (3.684, 'B'), (3.684, 'C'), (3.947, 'A'), (3.947, 'B'), (3.947, 'C'), (4.211, 'A'), (4.211, 'B'), (4.211, 'C'), (4.474, 'A'), (4.474, 'B'), (4.474, 'C'), (4.737, 'A'), (4.737, 'B'), (4.737, 'C'), (5.0, 'A'), (5.0, 'B'), (5.0, 'C')}
    U_index :

In [12]:
tlist = [t for t in r1.model.alltime]
len(tlist), C_data.shape

(20, (20, 3))

In [ ]:
r1.run_opt()

In [ ]:
print("The estimated parameters are:")
# Display the results
r1.results.show_parameters

In [ ]:
fig, ax = plt.subplots()
for c in ['A', 'B', 'C']:
    ax.scatter(r1.results.Cm.index, r1.results.Cm[c], label = c)
    ax.plot(r1.results.Z.index, r1.results.Z[c], label = c)
ax.legend()

In [ ]:
dof = r1.results.Cm.shape[0] * 3 - 2
cov_mat = r1.results.objective / dof * r1.results.parameter_covariance
cov_mat